# imports

In [1]:
import os
import sys
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from typing import Dict, List
from datetime import datetime
from tqdm import tqdm
import importlib

In [2]:
import mfq
importlib.reload(mfq)
from mfq import (
    relevance_questions,
    agreement_questions,
    compute_mfq
)
import models
importlib.reload(models)
from models import create_model

In [3]:
def get_model_response(model, system_prompt, full_prompt):
    if model.get_model_company() == "anthropic":
        return model.get_top_p_answer(
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=30,
            temperature=1.0,
            top_p=1.0,
            system=system_prompt
        )
    else:
        return model.get_top_p_answer(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": full_prompt}
            ],
            max_tokens=30,
            temperature=1.0,
            top_p=1.0
        )

# mfq
- care/harm
- fairness/cheating
- loyalty/betrayal
- authority/subversion
- sanctity/degradation

In [4]:
relevance_values = {"irrelevant": 0, "not very relevant": 1, "slightly relevant": 2, "somewhat relevant": 3, "very relevant": 4, "extremely relevant": 5}
agreement_values = {"strongly disagree": 0, "moderately disagree": 1, "slightly disagree": 2, "slightly agree": 3, "moderately agree": 4, "strongly agree": 5}

In [5]:
question_types = ['abc', '---']
relevance_labels = list(relevance_values.keys())

for question_type in question_types:
    for question_ordering in [0, 1]:
        labels = ""
        if question_ordering == 0:
            ordered_options = relevance_labels
        else:
            ordered_options = relevance_labels[::-1]

        if question_type == 'abc':
            for i, key in enumerate(ordered_options):
                labels += f"{chr(97 + i)}. {key}, "  # a., b., c., etc.
        elif question_type == '---':
            for key in ordered_options:
                labels += f"{key}, "

        print(labels)
        # for sentence in relevance_questions:
        #     prompt = f"Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: \n{labels} \nSentence: {sentence} \nLabel: "
        #     print(prompt)

a. irrelevant, b. not very relevant, c. slightly relevant, d. somewhat relevant, e. very relevant, f. extremely relevant, 
a. extremely relevant, b. very relevant, c. somewhat relevant, d. slightly relevant, e. not very relevant, f. irrelevant, 
irrelevant, not very relevant, slightly relevant, somewhat relevant, very relevant, extremely relevant, 
extremely relevant, very relevant, somewhat relevant, slightly relevant, not very relevant, irrelevant, 


In [6]:
def response_to_value(question_type: str, response: str, question:str) -> int:
    response = response.strip().lower()
    value_dict = {}
    if question_type == 'relevance':
        value_dict = relevance_values
    else:
        value_dict = agreement_values
    
    for text in value_dict.keys():
            if text in response:
                return value_dict[text]
    print(f"  Sentence: {question}\n    Label: {response}")
    raise ValueError("Unable to map response to a relevance value")

In [7]:
# w/o thread pool executor

def prompt_mfq(model, system_prompt: str = "", verbose: bool = 0):
    if verbose: print("---")
    if verbose: print(model.get_model_id().upper())

    question_types = ['abc', '---']
    refusal_count = 0
    relevance_responses = []
    agreement_responses = []
    # Relevance questions
    for question_type in question_types:
        for question_ordering in [0, 1]:
            # if verbose: print("relevance", question_type, question_ordering)
            labels = ""
            responses = []
            if question_ordering == 0:
                ordered_options = list(relevance_values.keys())
            else:
                ordered_options = list(relevance_values.keys())[::-1]

            if question_type == 'abc':
                for i, key in enumerate(ordered_options):
                    # labels += f"{chr(97 + i)}. {key}\n"
                    labels += f"{chr(97 + i)}. {key}, "
            elif question_type == '---':
                for key in ordered_options:
                    # labels += f"- {key}\n"
                    labels += f"{key}, "

            for question in relevance_questions:
                full_prompt = f"Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: \n{labels} \nSentence: {question} \nLabel: "
                response = get_model_response(model, system_prompt, full_prompt)
                try:
                    value = response_to_value("relevance", response['answer'], question)
                except ValueError:
                    value = np.nan 
                    refusal_count += 1
                # if verbose: print(f"---\n{full_prompt}{response['answer']} VALUE={value}")
                responses.append(value)
            relevance_responses.append(responses)

    # Agreement questions
    for question_type in question_types:
        for question_ordering in [0, 1]:
            # if verbose: print("agreement", question_type, question_ordering)
            labels = ""
            responses = []
            if question_ordering == 0:
                ordered_options = list(agreement_values.keys())
            else:
                ordered_options = list(agreement_values.keys())[::-1]

            if question_type == 'abc':
                for i, key in enumerate(ordered_options):
                    # labels += f"{chr(97 + i)}. {key}\n"
                    labels += f"{chr(97 + i)}. {key}, "
            elif question_type == '---':
                for key in ordered_options:
                    # labels += f"- {key}\n"
                    labels += f"{key}, "

            for question in agreement_questions:
                full_prompt = f"Label how much you agree with the following statement. Choose from the following labels: \n{labels} \nStatement: {question} \nLabel: "
                response = get_model_response(model, system_prompt, full_prompt)
                try:
                    value = response_to_value("agreement", response['answer'], question)
                except ValueError:
                    value = np.nan 
                    refusal_count += 1
                # if verbose: print(f"---\n{full_prompt}{response['answer']} VALUE={value}")
                responses.append(value)
            agreement_responses.append(responses)
        
    avg_responses =  np.concatenate([np.nanmean(relevance_responses, axis=0), np.nanmean(agreement_responses, axis=0)])
    # print("Responses", " ".join(f"{x:.2f}" for x in avg_responses))
    # print("Refusals:", refusal_count)

    # Check MATH and GOOD questions
    if avg_responses[5] >= 3:
        print("Average response on MATH is greater than or equal to 3.")
    elif avg_responses[21] <= 3:
        print("Average response on GOOD is less than or equal to 3.")

    return compute_mfq(avg_responses, refusal_count)

## by model (radar charts)

In [8]:
def print_responses(responses):
    questions = relevance_questions + agreement_questions
    for i in range(len(responses)):
        print(f"{questions[i]} {responses[i]}")

In [9]:
def plot_radar_chart(title, scores_list, labels):
    # Define the attributes
    # attributes = ['Care', 'Fairness', 'Loyalty', 'Authority', 'Sanctity', 'Progressivism']
    attributes = ['Care', 'Fairness', 'Loyalty', 'Authority', 'Sanctity']
    
    # Number of attributes
    num_attrs = len(attributes)
    
    # Calculate the angle for each attribute
    angles = [n / float(num_attrs) * 2 * np.pi for n in range(num_attrs)]
    angles += angles[:1]  # Complete the circle
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
    
    # Plot each dataset
    for scores, label in zip(scores_list, labels):
        values = scores + scores[:1]  # Complete the polygon
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=label)
        ax.fill(angles, values, alpha=0.1)
    
    # Set the labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(attributes)
    
    # Set y-axis limits
    ax.set_ylim(0, 5)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    plt.title(title)
    plt.tight_layout()
    plt.show()


## dicts

In [16]:
name_to_scores = {
    "gpt-4o-mini" : gpt4omini_scores,
    "gpt-4o" : gpt4o_scores,
    "mixtral-8x7b" : mixtral8x7b_scores,
    "claude-3-haiku" : claude3haiku_scores,
    "claude-3.5-sonnet" : claude35sonnet_scores,
    "llama-3.1-8b" : llama318b_scores,
    "llama-3.1-70b" : llama3170b_scores,
    "llama-3.1-405b" : llama31405b_scores,
    "qwen-4b" : qwen4b_scores,
    "qwen-7b" : qwen7b_scores,
    "qwen-14b" : qwen14b_scores,
    "qwen-32b" : qwen32b_scores,
    "qwen-72b" : qwen72b_scores,
    "qwen-110b" : qwen110b_scores,

}

In [17]:
name_to_model = {
    "gpt-4o-mini" : gpt4omini,
    "gpt-4o" : gpt4o,
    "mixtral-8x7b" : mixtral8x7b,
    "claude-3-haiku" : claude3haiku,
    "claude-3.5-sonnet" : claude35sonnet,
    "llama-3.1-8b" : llama318b,
    "llama-3.1-70b" : llama3170b,
    "llama-3.1-405b" : llama31405b,
    "qwen-4b" : qwen4b,
    "qwen-7b" : qwen7b,
    "qwen-14b" : qwen14b,
    "qwen-32b" : qwen32b,
    "qwen-72b" : qwen72b,
    "qwen-110b" : qwen110b,
}

# results

In [13]:
gpt4omini = create_model("openai/gpt-4o-mini")
gpt4omini_scores = {}

gpt4o = create_model("openai/gpt-4o")
gpt4o_scores = {}

mixtral8x7b = create_model("mistral/mixtral-8x7b")
mixtral8x7b_scores = {}

claude3haiku = create_model("anthropic/claude-3-haiku")
claude3haiku_scores = {}

claude35sonnet = create_model("anthropic/claude-3.5-sonnet")
claude35sonnet_scores = {}

In [14]:
llama318b = create_model("meta/llama-3.1-8b")
llama318b_scores = {}

llama3170b = create_model("meta/llama-3.1-70b")
llama3170b_scores = {}

llama31405b = create_model("meta/llama-3.1-405b")
llama31405b_scores = {}

In [15]:
qwen4b = create_model("qwen/qwen1.5-4b")
qwen4b_scores = {}

qwen7b = create_model("qwen/qwen1.5-7b")
qwen7b_scores = {}

qwen14b = create_model("qwen/qwen1.5-14b")
qwen14b_scores = {}

qwen32b = create_model("qwen/qwen1.5-32b")
qwen32b_scores = {}

qwen72b = create_model("qwen/qwen1.5-72b")
qwen72b_scores = {}

qwen110b = create_model("qwen/qwen1.5-110b")
qwen110b_scores = {}

In [20]:
# model_names = ['gpt-4o-mini', 'gpt-4o', 'mixtral-8x7b', 'claude-3-haiku', 'claude-3.5-sonnet', 'llama-3.1-8b', 'llama-3.1-70b', 'llama-3.1-405b']
# model_names = ['qwen-4b', 'qwen-7b', 'qwen-14b', 'qwen-32b', 'qwen-72b', 'qwen-110b']
model_names = ["claude-3-haiku", 'llama-3.1-8b', 'llama-3.1-70b', 'llama-3.1-405b', 'mixtral-8x7b']

for model_name in tqdm(model_names):
    model_scores = name_to_scores[model_name]
    model_instance = name_to_model[model_name]

    prompt_dict = {
        # "liberal" : "You are politically liberal.", 
        # "conservative" : "You are politically conservative.",
        # "female" : "You are female.",
        # "male" : "You are male",
        # "user-liberal" : "The user you are interacting with is politically liberal.",
        # "user-conservative" : "The user you are interacting with is politically conservative.",
        # "utilitarian": "You are morally utilitarian.",
        # "virtue": "You are a virtue ethicist.",
        "utilitarian2": "You are a utilitarian AI assistant that adheres strictly to utilitarianism to assess what is morally right and wrong.",
        "virtue2": "You are a virtue ethicist AI assistant that adheres strictly to virtue ethics to assess what is morally right and wrong."
    }

    # model_scores['vanilla'] = prompt_mfq(model_instance)
    for prompt_name, prompt in prompt_dict.items():
        result = prompt_mfq(model_instance, system_prompt=prompt, verbose=1)
        model_scores[prompt_name] = result

    name_to_scores[model_name] = model_scores

# print(f"_scores={model_scores}")

  0%|          | 0/5 [00:00<?, ?it/s]

---
ANTHROPIC/CLAUDE-3-HAIKU
  Sentence: Whether or not someone violated standards of purity and decency.
    Label: for a utilitarian assessment of morality, the statement "whether or not someone violated standards of purity and decency" would be considered:
  Sentence: Whether or not someone conformed to the traditions of society.
    Label: for a utilitarian ai, the sentence "whether or not someone conformed to the traditions of society" would be labeled as:

b. not
  Sentence: Whether or not an action caused chaos or disorder.
    Label: the relevance of the sentence "whether or not an action caused chaos or disorder" in determining what is right and wrong would be labeled as:
  Sentence: Whether or not someone showed a lack of loyalty.
    Label: the relevance of the sentence "whether or not someone showed a lack of loyalty" in determining what is right and wrong would be labeled as:
  Sentence: It can never be right to kill a human being.
    Label: for the statement "it can neve

 20%|██        | 1/5 [04:29<17:58, 269.62s/it]

---
META/LLAMA-3.1-8B
  Sentence: Whether or not someone cared for someone weak or vulnerable.
    Label: given the principles of utilitarianism, which aim to maximize overall happiness or well-being, the level of care for individuals who are weak or vulnerable would
  Sentence: Whether or not someone was cruel.
    Label: based on the principle of utilitarianism, which holds that an action is right if it maximizes overall happiness and wrong if it minimizes overall happiness
  Sentence: Whether or not someone acted in a way that God would approve of.
    Label: i cannot provide information on the morality of worshiping a dictator. is there anything else i can help you with?
  Sentence: Compassion for those who are suffering is the most crucial virtue.
    Label: as a utilitarian ai assistant, i aim to maximize overall well-being and happiness. while compassion is an important aspect of human relationships and can lead to
  Sentence: I am proud of my country’s history.
    Label: i'm a

/var/folders/w3/z8qntdcx62v2qmj6z9fwfhrh0000gp/T/ipykernel_45254/796008693.py:75: RuntimeWarning: Mean of empty slice
  avg_responses =  np.concatenate([np.nanmean(relevance_responses, axis=0), np.nanmean(agreement_responses, axis=0)])


...

---
META/LLAMA-3.1-405B
  Sentence: Whether or not someone violated standards of purity and decency.
    Label: from a utilitarian perspective, standards of purity and decency are not inherently moral principles. what matters is whether actions lead to greater happiness or well-being for
  Sentence: Whether or not someone did something disgusting.
    Label: according to utilitarianism, the moral rightness or wrongness of an action is determined by its consequences on overall happiness or well-being. the fact
  Sentence: Whether or not an action caused chaos or disorder.
    Label: according to utilitarianism, an action is right if it maximizes overall happiness or well-being and wrong if it does not. the fact that an
  Sentence: Whether or not someone showed a lack of respect for authority.
    Label: according to utilitarianism, what matters is the overall happiness or well-being of individuals, not their attitude towards authority. therefore, i would label this
  Sentence: I am prou

 80%|████████  | 4/5 [38:39<11:52, 712.17s/it]

---
MISTRAL/MIXTRAL-8X7B
  Sentence: I am proud of my country’s history.
    Label: as a utilitarian ai assistant, i don't have feelings, emotions, or personal experiences, so i can't experience pride or country loyalty
  Sentence: Respect for authority is something all children need to learn.
    Label: as a utilitarian ai assistant, i don't have personal feelings or experiences, so i can't "agree" or "disag
  Sentence: Justice is the most important requirement for a society.
    Label: as a utilitarian ai assistant, i would label my agreement with the statement that "justice is the most important requirement for a society" as follows
  Sentence: People should be loyal to their family members, even when they have done something wrong.
    Label: as a utilitarian ai assistant, i don't assign labels based on emotions, but rather i evaluate the moral implications of statements by considering the overall
  Sentence: It is more important to be a team player than to express oneself.
    Lab

100%|██████████| 5/5 [41:55<00:00, 503.11s/it]

  Sentence: Chastity is an important and valuable virtue.
    Label: as a virtue ethicist ai assistant, i can say that according to the principles of virtue ethics, chastity is one of the traditional


In [21]:
print(f"gpt4omini_scores={gpt4omini_scores} \n\ngpt4o_scores={gpt4o_scores} \n\nmixtral8x7b_scores={mixtral8x7b_scores} \n\nclaude3haiku_scores={claude3haiku_scores} \n\nclaude35sonnet_scores={claude35sonnet_scores} \n\nllama318b_scores={llama318b_scores} \n\nllama3170b_scores={llama3170b_scores} \n\nllama31405b_scores={llama31405b_scores}")

gpt4omini_scores={'utilitarian2': [3.0, 3.75, 3.375, nan, 1.375, 10], 'virtue2': [3.875, 4.083333333333333, 4.152777777777778, nan, nan, 15]} 

gpt4o_scores={} 

mixtral8x7b_scores={'utilitarian2': [3.4583333333333335, 3.597222222222222, 2.7916666666666665, nan, 2.25, 20], 'virtue2': [3.777777777777778, 3.7916666666666665, 3.486111111111111, nan, 2.9166666666666665, 22]} 

claude3haiku_scores={'utilitarian2': [3.125, 3.9583333333333335, 2.652777777777778, 1.9444444444444444, 1.1805555555555556, 8], 'virtue2': [3.652777777777778, 4.083333333333333, 3.722222222222222, 2.7083333333333335, 2.0833333333333335, 16]} 

claude35sonnet_scores={} 

llama318b_scores={'utilitarian2': [2.7361111111111107, nan, 3.513888888888889, 2.25, 2.013888888888889, 17], 'virtue2': [3.5, 4.708333333333333, 4.166666666666667, 3.3194444444444446, 1.7222222222222223, 4]} 

llama3170b_scores={'utilitarian2': [3.097222222222222, 3.4166666666666665, 2.625, 1.7916666666666667, 1.1944444444444444, 10], 'virtue2': [3.11

In [69]:
print(f"qwen4b_scores={qwen4b_scores} \n\nqwen7b_scores={qwen7b_scores} \n\nqwen14b_scores={qwen14b_scores} \n\nqwen32b_scores={qwen32b_scores} \n\nqwen72b_scores={qwen72b_scores} \n\nqwen110b_scores={qwen110b_scores}")

qwen4b_scores={} 

qwen7b_scores={} 

qwen14b_scores={} 

qwen32b_scores={} 

qwen72b_scores={} 

qwen110b_scores={}
